In [6]:
import numpy as np
%run Dolphins_data.ipynb
import Functions

In [3]:
mu = np.zeros(2) # Define mean (mu) as a zero vector
cov_matrix = np.array([[1.0, 0.5], [0.5, 2.0]]) # Define covariance matrix
num_samples = 62  # Number of samples you want to generate

points = {l+1: np.random.multivariate_normal(mu, cov_matrix) for l in range(num_samples)}

In [4]:
Functions.grad(1,0,Y,points)

-37.58892958316781

In [5]:
Functions.function(points,Y)

KeyError: 0.0

In [7]:
(np.linalg.norm(points[Y[1][1]-1] - points[Y[1][2]-1]))**2

KeyError: 0.0

In [11]:
points[intY[1][1]-1]

NameError: name 'intY' is not defined

In [9]:
Y[1][1]

1.0